## from itertools import permutations 
import numpy as np
from google.cloud import storage
#hasht = spark.read.json('gs://bgse-datawarehousing-random-tweets/2019-02-26T00:00:30.657Z')

client=storage.Client()
bucket=client.bucket("bgse-datawarehousing-random-tweets")
json_list=[]
for blob in bucket.list_blobs(prefix=""):
    json_list.append("gs://bgse-datawarehousing-random-tweets/"+blob.name)  

# Read in the first 5 files
hasht= spark.read.json(json_list[0:6109])

In [54]:
#hasht.printSchema()

In [32]:
#hasht.rdd.filter(lambda r: r.entities.hashtags).collect()

In [ ]:
x=hasht.rdd \
    .filter(lambda o: o.entities is not None) \
    .map(lambda o: o.entities) \
    .filter(lambda o: len(o.hashtags) > 1) \
    .map(lambda o:[i.text.lower() for i in o.hashtags]) \
    .flatMap(lambda o:permutations(o,2)) \
    .map(lambda l: (l,1)) \
    .reduceByKey(lambda a, b: a+b) \
    .map(lambda l:(l[0][0], l[0][1], l[1])) \
    .collect()
    
    #.map(lambda o: o.price_each * o.quantity_ordered)\
   # .reduce(add)

In [47]:
#x

In [ ]:
from collections import Counter
c=Counter(elem[0] for elem in x)
len(c)

In [49]:
rows=np.array([c[i[0]] for i in x])
columns=np.array([c[i[1]] for i in x])
data=np.array([i[2] for i in x])
data

array([1, 9, 1, ..., 2, 1, 1])

In [50]:
#dict(c)

In [51]:
import scipy.sparse

sparse_mat=scipy.sparse.coo_matrix((data,(rows, columns)), shape=(len(c),len(c)))


In [52]:
scipy.sparse.save_npz("sparseMatrix.npz", sparse_mat, compressed=True)